In [10]:
import pandas as pd
import csv
import time
from sklearn.model_selection import train_test_split

import warnings
##visulation of NA
import missingno


# check version number
import imblearn
print(imblearn.__version__)

from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

0.8.0


In [7]:
# load data
train = pd.read_csv("train.csv", header = None)

test = pd.read_csv("test.csv", header = None)

#print(test.info())
#train.info()


In [8]:
# minor_size is number of classes less than 500 instances
minor_size = 0
grouped = train.groupby(train.columns[-1])
for k,group in grouped:
        if len(group.index) < 500:
            minor_size += 1
        #print ("Class label: ",k,end='')
        #print("  Number:",len(group.index), end='')
        #print(" (%.3f%%)" %  float(len(group.index)/len(train)*100))
minor_size

39

## Split Data

In [9]:
# exclude sample ID (1st column)
x_train_valid = train.iloc[:,1:-1].to_numpy()
y_train_valid = train.iloc[:,-1].to_numpy()

#split a validation set for hyper-parameter tuning
split_ratio = 0.2
x_train, x_val, y_train, y_val = train_test_split(x_train_valid, y_train_valid, test_size=split_ratio, random_state=1,stratify = y_train_valid) 

x_test = test.iloc[:,1:].to_numpy()
#y_test = test.iloc[:,-1].to_numpy()

### Apply SMOTE

In [5]:
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, SMOTE

# we only oversample during training
ada = ADASYN(random_state = 42, sampling_strategy = 'minority', n_neighbors=5)
bdsmt = BorderlineSMOTE(random_state = 42)
smt = SMOTE(random_state = 42)

x_ada_train = x_train.copy()
y_ada_train = y_train.copy()

x_bdsmt_train = x_train.copy()
y_bdsmt_train = y_train.copy()

x_smt_train = x_train.copy()
y_smt_train = y_train.copy()

# ada failed to resample all minority class at once, we manually resample minority classes(less than 500 instances)
for i in range(minor_size):
    x_ada_train, y_ada_train = ada.fit_resample(x_ada_train, y_ada_train)
    
    
x_bdsmt_train, y_bdsmt_train = bdsmt.fit_resample(x_bdsmt_train, y_bdsmt_train)
x_smt_train, y_smt_train = smt.fit_resample(x_smt_train, y_smt_train)



#print("ada size: ", len(x_ada_train))
#print("bdsmt size: ", len(x_bdsmt_train))
print("ada size: ", len(y_ada_train))
print("bdsmt size: ", len(y_bdsmt_train))
print("smt size: ", len(y_smt_train))



ada size:  25076
bdsmt size:  22222
smt size:  23128


## Modelling

### SVM

- SMOTE(boardline, smote, ada) data (sampling method)
    - svc
    - svclinear

In [11]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
from sklearn.utils import class_weight
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import AdaBoostClassifier
import time

In [9]:
Test_Accuracy_Data = pd.DataFrame(columns =  ['option','C','gamma','kernel',
                                              'random_state','degree','dual','loss',
                                              'Train ACC','Val ACC'])

## SVC

In [ ]:
trainStartTime = time.time()

option = 'option 1-svc'

C = 1.0

gamma = 'auto'

kernel = 'rbf'

degree = 3

random_state = 0


dual = True

loss = 'squared_hinge'



svmc = SVC(C=C, gamma=gamma, kernel = kernel, random_state = random_state, degree = degree)

model= svmc.fit(x_bdsmt_train, y_bdsmt_train)

Test_Accuracy_Data_One = pd.DataFrame(index = range(1),columns = ['option','C','gamma','kernel',
                                                                  'random_state','degree','dual','loss'
                                                                  'Train ACC','Val ACC'])

Test_Accuracy_Data_One.loc[:,'option'] = option
Test_Accuracy_Data_One.loc[:,'C'] = C
Test_Accuracy_Data_One.loc[:,'gamma'] = gamma
Test_Accuracy_Data_One.loc[:,'kernel'] = kernel            
Test_Accuracy_Data_One.loc[:,'random_state'] = random_state

Test_Accuracy_Data_One.loc[:,'degree'] = degree
Test_Accuracy_Data_One.loc[:,'dual'] = dual       
Test_Accuracy_Data_One.loc[:,'loss'] = loss


train_pre = svmc.predict(x_bdsmt_train)
train_acc = svmc.score(x_bdsmt_train, y_bdsmt_train)
train_acc1 = accuracy_score(y_bdsmt_train, train_pre)

predictions = svmc.predict(x_val)
acc = svmc.score(x_val, y_val)
acc1 = accuracy_score(y_val, predictions)

Test_Accuracy_Data_One.loc[:,'Train ACC'] = train_acc
Test_Accuracy_Data_One.loc[:,'Val ACC'] = acc 

Test_Accuracy_Data = Test_Accuracy_Data.append(Test_Accuracy_Data_One)
            
print(train_acc1)
print(acc1)

trainFinishTime = time.time()
print("Time spent on training is: " + str(trainFinishTime - trainStartTime) + " sec")

Test_Accuracy_Data

In [8]:
#grid search for hyperparameter tunning
from sklearn.model_selection import GridSearchCV

trainStartTime = time.time()

# function for acc
def acc_score(cl,x,y):
    pred = cl.predict(x_val)
    acc=cl.score(x_val,y_val)
    print(cl, acc)
    return acc


param = {'C': [0.1, 1, 10], 
              'gamma': [0.1, 0.01, 'scale'],
              'kernel': ['rbf','linear']} 

grid = GridSearchCV(SVC(random_state = 0), param, n_jobs=-1, scoring=acc_score, verbose=10)

# fitting the model for grid search
grid.fit(x_bdsmt_train, y_bdsmt_train)
pred=grid.predict(x_val)

#print out the best parameters
print(grid.best_params_)

trainFinishTime = time.time()
print("Time spent on training is: " + str(trainFinishTime - trainStartTime) + " sec")

## Linear SVC

In [ ]:
trainStartTime = time.time()

option = 'option 1-linearsvc'

C = 1.0

gamma = 'auto'

kernel = 'rbf'

degree = 3

random_state = 0


dual = True

loss = 'squared_hinge'


svmc = LinearSVC(C=C,random_state = random_state, dual = dual)


model= svmc.fit(x_bdsmt_train, y_bdsmt_train)

Test_Accuracy_Data_One = pd.DataFrame(index = range(1),columns = ['option','C','gamma','kernel',
                                                                  'random_state','degree','dual','loss'
                                                                  'Train ACC','Val ACC'])

Test_Accuracy_Data_One.loc[:,'option'] = option
Test_Accuracy_Data_One.loc[:,'C'] = C
Test_Accuracy_Data_One.loc[:,'gamma'] = gamma
Test_Accuracy_Data_One.loc[:,'kernel'] = kernel            
Test_Accuracy_Data_One.loc[:,'random_state'] = random_state

Test_Accuracy_Data_One.loc[:,'degree'] = degree
Test_Accuracy_Data_One.loc[:,'dual'] = dual       
Test_Accuracy_Data_One.loc[:,'loss'] = loss


train_pre = svmc.predict(x_bdsmt_train)
train_acc = svmc.score(x_bdsmt_train, y_bdsmt_train)
train_acc1 = accuracy_score(y_bdsmt_train, train_pre)

predictions = svmc.predict(x_val)
acc = svmc.score(x_val, y_val)
acc1 = accuracy_score(y_val, predictions)

Test_Accuracy_Data_One.loc[:,'Train ACC'] = train_acc
Test_Accuracy_Data_One.loc[:,'Val ACC'] = acc 

Test_Accuracy_Data = Test_Accuracy_Data.append(Test_Accuracy_Data_One)
            
print(train_acc1)
print(acc1)

trainFinishTime = time.time()
print("Time spent on training is: " + str(trainFinishTime - trainStartTime) + " sec")

Test_Accuracy_Data

In [ ]:
#grid search for hyperparameter tunning
from sklearn.model_selection import GridSearchCV

trainStartTime = time.time()

# function for acc
def acc_score(cl,x,y):
    pred = cl.predict(x_val)
    acc=cl.score(x_val,y_val)
    print(cl, acc)
    return acc

# defining parameter range
param = {'C': [0.1, 1, 10],
        'dual' : [True, False]} 

grid = GridSearchCV(LinearSVC(random_state = 0), param, n_jobs=-1, scoring=acc_score, verbose=10)

# fitting the model for grid search
grid.fit(x_bdsmt_train, y_bdsmt_train)
pred=grid.predict(x_val)

#print out the best parameters
print(grid.best_params_)

trainFinishTime = time.time()
print("Time spent on training is: " + str(trainFinishTime - trainStartTime) + " sec")

## Output Final Result


In [ ]:
# test on the test data
predictions=svmc.predict(x_test)

with open("svm_out.csv","w", newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Category"])
    test_id = 0
    for pred in predictions:
        writer.writerow([test_id, pred])
        test_id+=1